In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

2024-10-29 18:22:24.743579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 18:22:24.754614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 18:22:24.767509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 18:22:24.771230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 18:22:24.780581: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import pandas as pd
import numpy as np
import joblib
# Load the original "no smoke" data
no_smoke_data = pd.read_csv('/home/hthnh/Desktop/project-khkt-thcs/noSmokeData.csv')  # Replace with actual file path if needed
no_smoke_data['Label'] = 0  # Label for no-smoke data

# Define the number of synthetic smoke samples and their characteristic ranges
n_smoke_samples = 1000  # Number of synthetic smoke data points
smoke_data = pd.DataFrame({
    'time': pd.date_range(start='2023-01-01', periods=n_smoke_samples, freq='S'),  # Time range for smoke data
    'Co': np.random.uniform(50, 200, n_smoke_samples),  # Elevated CO levels
    'VOC': np.random.uniform(100, 1000, n_smoke_samples),  # Elevated VOC levels
    'Temp': np.random.uniform(no_smoke_data['Temp'].min(), no_smoke_data['Temp'].max(), n_smoke_samples),
    'Hum': np.random.uniform(no_smoke_data['Hum'].min(), no_smoke_data['Hum'].max(), n_smoke_samples),
    'Label': 1  # Label for smoke data
})

# Combine no-smoke and synthetic smoke data
combined_data = pd.concat([no_smoke_data, smoke_data], ignore_index=True)

# Select the features for normalization
features = ['Co', 'VOC', 'Temp', 'Hum']

# Initialize and fit the scaler on the combined data
scaler = joblib.load('/home/hthnh/Desktop/project-khkt-thcs/model/scaler.pkl')

combined_data[features] = scaler.transform(combined_data[features])

# Save the normalized combined data with labels to a CSV file
combined_data.to_csv('normalized_no_smoke_and_smoke_data_with_labels.csv', index=False)
data = combined_data
# Display the first few rows of the normalized data


/tmp/ipykernel_42082/2038843518.py:11: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  'time': pd.date_range(start='2023-01-01', periods=n_smoke_samples, freq='S'),  # Time range for smoke data


In [2]:
data = pd.read_csv('/home/hthnh/Desktop/project-khkt-thcs/model/processed_fit.csv')

In [3]:
X = data[['Co', 'VOC', 'Temp', 'Hum']]
y = data['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
base_model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification (normal vs smoke)
])

In [5]:
base_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
base_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


In [10]:
base_model.save('base_smoke_model.keras')

In [4]:
from tensorflow.keras.models import load_model
base_model = load_model('base_smoke_model.keras')

I0000 00:00:1730200981.870986   42082 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-29 18:23:01.895567: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
# Freeze all layers except the last layer (or last few layers if you want to fine-tune more)
for layer in base_model.layers[:-1]:  # Adjust this to control how many layers to freeze
    layer.trainable = False


from tensorflow.keras.layers import Dense

# Remove the last layer
base_model.pop()

# Add new layers with unique names
base_model.add(Dense(64, activation='relu', name='dense_finetune_2'))  # Unique name for added hidden layer
base_model.add(Dense(1, activation='sigmoid', name='dense_output_finetune'))  # Unique name for output layer

# Compile the model with a low learning rate for fine-tuning
base_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Compile the model with a low learning rate for fine-tuning
base_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = base_model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), callbacks=[early_stopping])



Epoch 1/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9752 - loss: 0.5360 - val_accuracy: 0.9818 - val_loss: 0.4052
Epoch 2/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9726 - loss: 0.3757 - val_accuracy: 0.9838 - val_loss: 0.2983
Epoch 3/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9671 - loss: 0.2849 - val_accuracy: 0.9859 - val_loss: 0.2302
Epoch 4/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9736 - loss: 0.2211 - val_accuracy: 0.9859 - val_loss: 0.1835
Epoch 5/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9708 - loss: 0.1866 - val_accuracy: 0.9859 - val_loss: 0.1503
Epoch 6/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9742 - loss: 0.1529 - val_accuracy: 0.9859 - val_loss: 0.1261
Epoch 7/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9765 - loss: 0.1286 - val_accuracy: 0.9879 - val_loss: 0.1073
Epoch 8/500
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9781 - loss: 0.1124 - val_accuracy: 0.9879 - v

In [8]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
# Example new sensor readings (CO, VOC, Temp, Hum)
new_data = np.array([[40, 50, 30.2, 72.5]])  # Example values for testing
scaler = joblib.load('/home/hthnh/Desktop/project-khkt-thcs/model/scaler.pkl')
# Assuming you used a scaler for normalization during training
new_data_scaled = scaler.transform(new_data)  # Normalize the new data


/home/hthnh/.local/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
# Make predictions
prediction = base_model.predict(new_data_scaled)

# Interpret the prediction (assuming binary classification: 0 = normal, 1 = smoke)
if prediction[0][0] > 0.5:
    print("Smoke detected")
else:
    print("Normal environment")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Normal environment
